In [1]:
import time
import json
import bz2
import pandas as pd
#import jsonlines

In [2]:
###  IMPORTING DATA ###

# Define the path where the data is located

Path = "../quotes-2020.json.bz2" #goes out of the current folder


In [3]:
import pandas as pd

def process_chunk(chunk):
        print(f'Processing chunk with {len(chunk)} rows')
        print(chunk.columns)

with pd.read_json(Path, lines=True, compression='bz2', chunksize=1_000_000) as df_reader:
    dfs = []
    for chunk in df_reader:
        process_chunk(chunk)
        dfs.append(chunk)
        #chunk.to_json(compression='bz2')

Processing chunk with 1000000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 1000000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 1000000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 1000000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 1000000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 244449 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='ob

In [4]:
chunk.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
5000000,2020-03-06-047752,"People get killed in wildlife collisions, most...",Dave Marsden,[Q5240722],2020-03-06 17:20:36,3,"[[Dave Marsden, 0.6329], [None, 0.3671]]",[http://www.washingtontimes.com/news/2020/mar/...,E
5000001,2020-01-31-070691,People my age and older have to figure out how...,Michelle Buteau,[Q54861496],2020-01-31 17:47:00,2,"[[Michelle Buteau, 0.8651], [None, 0.1349]]",[https://www.nbcnews.com/news/nbcblk/michelle-...,E
5000002,2020-03-26-047932,People now have time on their hands and they w...,Gary Gray,"[Q21228842, Q3758498, Q5525157, Q5525158, Q552...",2020-03-26 19:43:00,1,"[[Gary Gray, 0.8611], [None, 0.1389]]",[http://capenews.net/sandwich/news/business-is...,E
5000003,2020-01-10-061824,People should not recourse to violent methods ...,Peter Machado,[Q1385539],2020-01-10 03:04:23,1,"[[Peter Machado, 0.9013], [None, 0.0987]]",[https://www.newkerala.com/news/2020/4460.htm],E


In [5]:
len(dfs)

6

In [8]:
pd.concat(dfs)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E
1,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
2,2020-02-10-000142,... He (Madhav) also disclosed that the illega...,None,[],2020-02-10 23:45:54,1,"[[None, 0.8926], [Prakash Rai, 0.1074]]",[https://indianexpress.com/article/business/ec...,E
3,2020-02-15-000053,"... [ I ] f it gets to the floor,",None,[],2020-02-15 14:12:51,2,"[[None, 0.581], [Andy Harris, 0.4191]]",[https://patriotpost.us/opinion/68622-trump-bu...,E
4,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
...,...,...,...,...,...,...,...,...,...
5244444,2020-02-24-080186,"you're seeing a young team that's maturing, th...",Brendan Whittet,[Q18115465],2020-02-24 05:00:28,1,"[[Brendan Whittet, 0.7077], [None, 0.2923]]",[http://feeds.browndailyherald.com/~r/BrownDai...,E
5244445,2020-02-07-122251,"You're talking about African-Americans, right?...",Barry Michael Cooper,[Q3635235],2020-02-07 00:00:00,1,"[[Barry Michael Cooper, 0.5605], [None, 0.4395]]",[https://www.villagevoice.com/2020/02/07/1980-...,E
5244446,2020-02-27-098715,You've got to own the team. I see Broosky and ...,None,[],2020-02-27 05:59:00,1,"[[None, 0.8899], [Trent Barrett, 0.0539], [Mic...",[https://www.foxsports.com.au/nrl/nrl-premiers...,E
5244447,2020-02-04-118820,You've got to sometimes take that leap of fait...,Brad Gushue,[Q896796],2020-02-04 14:47:00,10,"[[Brad Gushue, 0.706], [None, 0.2919], [John E...",[http://timescolonist.com/in-the-rings-steski-...,E
